In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.layers

In [3]:
from tensorflow.examples.tutorials.mnist import input_dataini

In [5]:
mnist=input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
# generating the discriminator fucntion

def dicriminator(x_image, reuse=False):
    if(reuse):
        tf.get_variable_scope().reuse_variables()
        
    #defining weights
    
    d_w1 = tf.get_variable('d_w1', [5, 5, 1, 32], initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #defining wbias
    
    
    d_b1 = tf.get_variable('d_b1', [32], initializer=tf.constant_initializer(0))
    
    